3 methods to implement autoencoder in tf 2

In [1]:
# Load dataset
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
x = x.reshape((x.shape[0], -1))
x = x.astype('float32') / 255.0

trainloader = tf.data.Dataset.from_tensor_slices((x, x)).shuffle(70000).batch(256)

In [2]:
# First implementation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense

class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.enc = Sequential([
            InputLayer(input_shape=(784)),
            Dense(500, activation='relu'),
            Dense(500, activation='relu'),
            Dense(2000, activation='relu'),
            Dense(10),
            ])

    def call(self, x):
        return self.enc(x)
    
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.dec = Sequential([
            InputLayer(input_shape=(10)),
            Dense(2000, activation='relu'),
            Dense(500, activation='relu'),
            Dense(500, activation='relu'),
            Dense(784, activation='sigmoid'),
            ])

    def call(self, z):
        return self.dec(z)

class AE(object):
    def __init__(self):
        self.enc = Encoder()
        self.dec = Decoder()
        self.optim = tf.keras.optimizers.Adam()
            
    def train(self):
        for epoch in tqdm(range(10)):
            epoch_loss = []
            for x_batch, _ in trainloader:
                with tf.GradientTape() as tape:
                    z = self.enc(x_batch)
                    x_rec = self.dec(z)
                    batch_loss = tf.keras.losses.BinaryCrossentropy()(x_batch, x_rec)
                             
                t_vars = self.enc.trainable_variables + self.dec.trainable_variables
                enc_grads = tape.gradient(batch_loss, t_vars)
                self.optim.apply_gradients(zip(enc_grads, t_vars))
                epoch_loss.append(batch_loss) 
            print('epoch_loss:{:.4f}'.format(tf.reduce_mean(epoch_loss).numpy()))
            
ae = AE()
ae.train()

 10%|████████▎                                                                          | 1/10 [00:04<00:37,  4.15s/it]

epoch_loss:0.2139


 20%|████████████████▌                                                                  | 2/10 [00:08<00:32,  4.08s/it]

epoch_loss:0.1306


 30%|████████████████████████▉                                                          | 3/10 [00:11<00:28,  4.01s/it]

...

epoch_loss:0.0998


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:34<00:03,  3.74s/it]

epoch_loss:0.0987


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:38<00:00,  3.81s/it]

epoch_loss:0.0978


In [3]:
# Second implementation
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(500, activation='relu', input_shape=(784,)))
model.add(Dense(500, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(10))
model.add(Dense(2000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(784, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())
model.fit(trainloader, epochs=10, verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 500)               392500    
_________________________________________________________________
dense_9 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_10 (Dense)             (None, 2000)              1002000   
_________________________________________________________________
dense_11 (Dense)             (None, 10)                20010     
_________________________________________________________________
dense_12 (Dense)             (None, 2000)              22000     
_________________________________________________________________
dense_13 (Dense)             (None, 500)               1000500   
_________________________________________________________________
dense_14 (Dense)             (None, 500)              

In [4]:
# Third implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense

class AE(tf.keras.Model):
    def __init__(self):
        super(AE, self).__init__()
        self.enc = tf.keras.Sequential([
            InputLayer(input_shape=(784,)),
            Dense(500, activation='relu'),
            Dense(500, activation='relu'),
            Dense(2000, activation='relu'),
            Dense(10),
        ])
        
        self.dec = tf.keras.Sequential([
            InputLayer(input_shape=(10,)),
            Dense(2000, activation='relu'),
            Dense(500, activation='relu'),
            Dense(500, activation='relu'),
            Dense(784, activation='sigmoid'),
        ])
          
    def encode(self, x):
        return self.enc(x)

    def decode(self, z):
        return self.dec(z)
    
    def call(self, x): 
        z = self.enc(x) 
        return self.dec(z)

ae = AE() 
ae.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())
ae.fit(trainloader, epochs=10, verbose=1)   

Epoch 1/10
274/274 [==============================] - 3s 11ms/step - loss: 0.2141
Epoch 2/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1283
Epoch 3/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1129
Epoch 4/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1077
Epoch 5/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1047
Epoch 6/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1026
Epoch 7/10
274/274 [==============================] - 2s 8ms/step - loss: 0.1009
Epoch 8/10
274/274 [==============================] - 2s 8ms/step - loss: 0.0995
Epoch 9/10
274/274 [==============================] - 2s 8ms/step - loss: 0.0985
Epoch 10/10
274/274 [==============================] - 2s 8ms/step - loss: 0.0975
